In [1]:
import logging

import hydra

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

In [2]:
with hydra.initialize(version_base=None, config_path="./config"):
    cfg = hydra.compose(config_name="config", overrides={"dataset.generate_csv=True"})
dataset = hydra.utils.instantiate(cfg.dataset)

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2Model: ['lm_head.weight', 'lm_head.bias']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Processing ECG / EDA / Label:   0%|          | 0/20 [00:00<?, ?it/s]

In [3]:
dataset[0]

{'wav': tensor([[ 0.0209, -0.0029,  0.0764,  ..., -0.2665,  0.0813, -0.0492],
         [ 0.0212, -0.0047,  0.0786,  ..., -0.2719,  0.0819, -0.0524],
         [ 0.0191, -0.0033,  0.0742,  ..., -0.2709,  0.0827, -0.0527],
         ...,
         [-0.0013, -0.0075,  0.0835,  ..., -0.2638,  0.0836, -0.0525],
         [ 0.0038, -0.0057,  0.0694,  ..., -0.2613,  0.0846, -0.0388],
         [-0.0020, -0.0101,  0.0656,  ..., -0.2671,  0.0870, -0.0373]]),
 'txt': None,
 'emotion': tensor(1),
 'valence': tensor(1.7000),
 'arousal': tensor(4.)}

In [4]:
from pathlib import Path

segment_id = dataset.total_df.segmend_id.iloc[250]
session, script_type, speaker = segment_id.split("_")

prefix = Path(dataset.wav_txt_path_fmt.format(session[-2:], script_type))
wav_path = prefix / f"{segment_id}.wav"
txt_path = prefix / f"{segment_id}.txt"

In [5]:
eda_path = dataset.eda_path_fmt.format(session[-2:], speaker[0])

In [9]:
txt_path

PosixPath('data/KEMDy19/wav/Session01/Sess01_script06/Sess01_script06_M008.txt')

In [7]:
with open(txt_path, mode="r") as f:
    a = f.readlines()

In [8]:
a

['오 자 시작한다 좋다 좋다 옳다 옳다 아싸 아싸 맞다 번호가 하나 맞았고.\n']

In [6]:
from scipy.io import wavfile
from transformers import Wav2Vec2FeatureExtractor, Wav2Vec2Model, Wav2Vec2Config

In [7]:
sampling_rate, data = wavfile.read(wav_path)

In [8]:
fe = Wav2Vec2FeatureExtractor()
wav_vector = fe(data, sampling_rate=sampling_rate, return_tensors="pt")

In [9]:
wav_vector["input_values"].shape

torch.Size([1, 172464])